In [1]:
# Import modules and subroutines:

include("../subroutines/Subroutines.jl");

In [2]:
# Load the chemical data into julia as an array of chemical data structs:
# (one struct obtained for each molecular geometry as set up in the config file)

cdata_list = ReadIn("../datasets/pyscf_data/h2o_sto-3g_080222%000756.hdf5");
chemical_data = cdata_list[1]
PrintChemData(chemical_data)

Molecule name: h2o
Basis set: sto-3g
Molecular geometry: O 0.0 0.0 0.0; H 1.0 0.0 0.0; H 0.0 1.0 0.0; 
RHF energy: -74.96117113786774
FCI energy: -75.01927599660624


In [30]:
subspace1 = GenSubspace(chemical_data, 40, psi_maxdim=3, sweep_num=10, sweep_noise=(1e-6,1e-8,0.0), thresh="inversion");

subspace1.ord_list = [collect(1:subspace1.chem_data.N_spt) for i=1:subspace1.mparams.M]

GenStates!(subspace1, ovlp_opt=true, weight=10.0, verbose=true)

GenSubspaceMats!(subspace1, verbose=true)

display(round.(subspace1.S_mat,digits=3))

display(round.(inv(subspace1.S_mat),digits=3))

Generating states:
Progress: [40/40] 
Done!
computing matrix elements:
Progress: [820/820] 
Done!


40×40 Matrix{Float64}:
  1.0     0.0     0.0     0.0     0.001  …   0.0     0.003  -0.0    -0.0
  0.0     1.0    -0.0    -0.0     0.0        0.04    0.0    -0.005   0.0
  0.0    -0.0     1.0     0.0     0.0       -0.13    0.0     0.007  -0.0
  0.0    -0.0     0.0     1.0     0.014     -0.0     0.026  -0.0     0.057
  0.001   0.0     0.0     0.014   1.0       -0.0     0.003   0.0     0.021
  0.001   0.0    -0.0     0.001  -0.0    …  -0.0    -0.022  -0.0    -0.022
  0.0     0.0     0.001  -0.0     0.0        0.01    0.0     0.071   0.0
 -0.0     0.0    -0.0    -0.003   0.005     -0.0     0.002   0.0     0.001
 -0.003   0.0     0.0     0.0     0.008     -0.0     0.01    0.0     0.03
  0.0    -0.001  -0.0    -0.0    -0.0       -0.002   0.0     0.009  -0.0
 -0.003   0.0    -0.0     0.009   0.022  …   0.0    -0.015  -0.0     0.025
  0.0     0.025  -0.016  -0.0     0.0        0.021   0.0     0.044  -0.0
  0.0     0.0     0.0     0.0    -0.0        0.0    -0.002  -0.0     0.012
  ⋮            

40×40 Matrix{Float64}:
  1.0     0.0    -0.0     0.0    -0.001  …  -0.0    -0.003   0.0     0.0
  0.0     1.005  -0.006  -0.0     0.0       -0.045  -0.0     0.006   0.0
 -0.0    -0.006   1.024  -0.0    -0.0        0.134   0.0    -0.007  -0.0
  0.0    -0.0    -0.0     1.006  -0.014      0.0    -0.027   0.0    -0.054
 -0.001   0.0    -0.0    -0.014   1.004      0.0    -0.003   0.0    -0.019
 -0.001   0.0    -0.0    -0.002  -0.001  …  -0.0     0.022  -0.0     0.024
  0.0    -0.0    -0.001   0.0    -0.0       -0.009  -0.0    -0.071   0.0
 -0.0    -0.0     0.0     0.002  -0.003      0.0    -0.002  -0.0    -0.004
  0.003  -0.0     0.0     0.0    -0.007      0.0    -0.01   -0.0    -0.03
  0.0     0.001   0.001   0.0     0.0        0.002  -0.0    -0.01    0.0
  0.003   0.0    -0.0    -0.006  -0.022  …  -0.0     0.016   0.0    -0.024
 -0.0    -0.025   0.014   0.0    -0.0       -0.017   0.0    -0.043  -0.0
 -0.0     0.0    -0.0     0.0     0.0       -0.0     0.002   0.0    -0.012
  ⋮            

In [73]:
subspace2 = GenSubspace(chemical_data, 1, psi_maxdim=3, sweep_num=4, sweep_noise=(1e1), thresh="inversion", sites=subspace1.sites);

subspace2.ord_list = [randperm(subspace2.chem_data.N_spt)]

println(subspace2.ord_list[1])

GenStates!(subspace2, verbose=true);

[1, 7, 4, 6, 2, 3, 5]
Generating states:
Progress: [1/1] 
Done!


In [74]:
ovlps=[]

for j=1:subspace1.mparams.M
    
    perm_psi_j = Permute(
        subspace1.psi_list[j],
        subspace1.sites, 
        subspace1.ord_list[j], 
        subspace2.ord_list[1], 
        tol=1e-10, 
        maxdim=1000, 
        spinpair=false, 
        locdim=4
    )
    
    #println("$(j)/$(subspace1.mparams.M)")
    
    push!(ovlps, inner(perm_psi_j, subspace2.psi_list[1]))
end

S_inv = inv(subspace1.S_mat)

total = 0.0

for g1=1:subspace1.mparams.M, g2=1:subspace1.mparams.M
    
    total += ovlps[g1]*ovlps[g2]*S_inv[g1,g2]
    
end

println(total)


0.9878954600446405
